# Mapeamento de possíveis candidaturas laranjas

No decorrer deste trabalho, iremos trabalhar com os dataset's do TSE referentes aos resultados das eleições municipais de 2020.

O art. 10, § 3º, da Lei das Eleições (Lei 9.504 de 30 de Setembro de 1997), determina que, pelo menos, 30% (trinta por cento) das candidaturas efetivamente lançadas por um partido político seja destinada ao gênero oposto ao da maioria.

O objetivo do nosso trabalho é mapear possíveis candidaturas laranjas, tendo como critério a ausência de votos em candidatos deferidos para disputar o pleito.

### Importando as dependências e configurando a exibição dos DataFrames

In [1]:
import geopandas as gdp
import pandas as pd
pd.set_option('display.max_columns', None)

Abaixo iremos importar os dados dos candidatos e dos resultados nominais por município.
Os dados podem ser acessados no link: https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

Entretanto, não iremos extrair todos os dados dos dataset's. Abaixo definiremos uma lista de colunas a ser utilizada, o que será útil futuramente no estudo.

In [2]:
colunasCandidatos = ['SQ_CANDIDATO','NR_IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO','DS_COR_RACA','DS_OCUPACAO']
colunasResultados = ['NM_MUNICIPIO','SG_UF','SQ_CANDIDATO','DS_CARGO','DS_SITUACAO_CANDIDATURA','DS_DETALHE_SITUACAO_CAND','SG_PARTIDO','QT_VOTOS_NOMINAIS']

In [3]:
dfCandidatos = pd.read_csv('dados/candidatos/consulta_cand_2020_BRASIL.csv', delimiter=';', encoding='latin-1', usecols = colunasCandidatos)

In [4]:
dfResultados = pd.read_csv('dados/resultados/votacao_candidato_munzona_2020_BRASIL.csv', delimiter = ';', encoding = 'latin-1', usecols = colunasResultados)

Agora, podemos observar os dados selecionados dos candidatos. Filtramos apenas os dados pertinentes para o nosso estudo. O nome de possíveis candidatos "laranjas" foram removidos, por não são informações pertinentes.

In [5]:
dfCandidatos.head()

,SQ_CANDIDATO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_COR_RACA,DS_OCUPACAO
0,210000883297,42,MASCULINO,ENSINO MÉDIO COMPLETO,BRANCA,AGRICULTOR
1,250001122285,49,MASCULINO,ENSINO MÉDIO COMPLETO,BRANCA,TÉCNICO DE QUÍMICA
2,250000983631,46,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,BRANCA,MECÂNICO DE MANUTENÇÃO
3,200000980650,34,MASCULINO,ENSINO MÉDIO COMPLETO,BRANCA,VEREADOR
4,110001047590,51,MASCULINO,SUPERIOR COMPLETO,BRANCA,BIÓLOGO


Abaixo, os dados filtrados referentes aos resultados dos candidatos.

In [6]:
dfResultados.head()

,SG_UF,NM_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,QT_VOTOS_NOMINAIS
0,CE,FORTALEZA,Vereador,60000936791,APTO,DEFERIDO,PSDB,29
1,AM,MANAUS,Vereador,40001182178,APTO,DEFERIDO,REPUBLICANOS,78
2,SP,CARDOSO,Vereador,250001266444,APTO,DEFERIDO,PSDB,11
3,SC,CRICIÚMA,Vereador,240000980207,APTO,DEFERIDO,PATRIOTA,2
4,MG,UBERABA,Vereador,130000757747,APTO,DEFERIDO,REPUBLICANOS,19


## Filtrando os candidatos que não possuíram nenhum voto

Neste momento, iremos preparar os dados para identificar o possíveis candidatos de fachada.

Mas antes disso, precisamos garantir que removemos candidatos não-aptos a disputar o pleito ou indeferidos pela Justiça Eleitoral, o que é uma justificativa para o não recebimento de votos.

In [7]:
dfResultados.query("DS_SITUACAO_CANDIDATURA == 'APTO' and DS_DETALHE_SITUACAO_CAND == 'DEFERIDO'", inplace = True)
dfResultados.head()

,SG_UF,NM_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,QT_VOTOS_NOMINAIS
0,CE,FORTALEZA,Vereador,60000936791,APTO,DEFERIDO,PSDB,29
1,AM,MANAUS,Vereador,40001182178,APTO,DEFERIDO,REPUBLICANOS,78
2,SP,CARDOSO,Vereador,250001266444,APTO,DEFERIDO,PSDB,11
3,SC,CRICIÚMA,Vereador,240000980207,APTO,DEFERIDO,PATRIOTA,2
4,MG,UBERABA,Vereador,130000757747,APTO,DEFERIDO,REPUBLICANOS,19


Após esta limpeza, iremos agrupar os números identificadores dos candidatos de acordo com a soma dos seus votos.

In [20]:
SeriesTotal = pd.Series(dfResultados.groupby('SQ_CANDIDATO').QT_VOTOS_NOMINAIS.sum())
len(SeriesTotal)

493058

Com esta relação direta, iremos filtrar os candidatos que tiveram um total de votos igual a zero.

In [17]:
dfResultados.sort_values(by = ['SQ_CANDIDATO'], inplace = True)

In [21]:
dfResultados.drop_duplicates(subset=['SQ_CANDIDATO'], inplace = True)
dfResultados.

,SG_UF,NM_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,DS_SITUACAO_CANDIDATURA,DS_DETALHE_SITUACAO_CAND,SG_PARTIDO,QT_VOTOS_NOMINAIS
257888,AC,ASSIS BRASIL,Vereador,10000641345,APTO,DEFERIDO,PSD,77
608047,AC,ASSIS BRASIL,Vereador,10000641347,APTO,DEFERIDO,PSD,23
143715,AC,ASSIS BRASIL,Vereador,10000641348,APTO,DEFERIDO,PSD,43
852413,AC,ASSIS BRASIL,Vereador,10000641349,APTO,DEFERIDO,PSD,59
642698,AC,ASSIS BRASIL,Vereador,10000641350,APTO,DEFERIDO,PSD,11
...,...,...,...,...,...,...,...,...
331272,TO,GOIATINS,Vereador,270001274471,APTO,DEFERIDO,SOLIDARIEDADE,23
811879,TO,CAMPOS LINDOS,Vereador,270001274474,APTO,DEFERIDO,SOLIDARIEDADE,14
57654,TO,JAÚ DO TOCANTINS,Prefeito,270001274657,APTO,DEFERIDO,PSC,992
677894,TO,ITACAJÁ,Vereador,270001275304,APTO,DEFERIDO,PT,116


Ou seja, a variável "candidatosSemVotos" referencia uma lista com todos os números sequenciais dos candidatos que não obtiveram nenhum voto na última eleição.

Por fim, limparemos o DataFrame de resultados com apenas os candidatos identificados, assim, poderemos relacionar de forma mais rápida e eficaz outros dados como sexo, cidade, idade e outros atributos.

In [10]:
candidatosSemVotos = pd.Series(candidatosSemVotos)
dfResultados[dfResultados.SQ_CANDIDATO in candidatosSemVotos]

TypeError: 'Series' objects are mutable, thus they cannot be hashed